In [1]:
import os
import pickle
import numpy as np
import time

from gensim.models import Word2Vec
from keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, hamming_loss, f1_score
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical

from models import get_rnn_model
from cm import multilabel_confusion_matrix
from data_process import get_embedding_matrix, data_generator, get_all_notes_labels, get_features, get_targets, get_gold_label_targets

# Customized Evaluation for keras model
class CustomEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = list(validation_data)

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = []
            for x in self.X_val:
                y = np.squeeze(self.model.predict_on_batch(x))
                y_pred.append(y)
            y_pred = np.concatenate(y_pred)
            y_pred_ham = y_pred > 0.5
            y_val = np.concatenate(self.y_val)
            roc = roc_auc_score(y_val, y_pred, average='micro')
            loss = log_loss(y_val, y_pred)
            ham = hamming_loss(y_val, y_pred_ham)
            sub = accuracy_score(y_val, y_pred_ham)
            f1 = f1_score(y_val, y_pred_ham, average='micro')
            print("Adiitional val metrics: - ROC-AUC: %.6f - Log-Loss: %.6f - Hamming-Loss: %.6f - Subset-Accuracy: %.6f - F1-Score: %.6f" % (roc, loss, ham, sub, f1))
            
def model_train(param, 
                notes_train, 
                labels_train, 
                up_notes_train, 
                up_labels_train, 
                gold_labels_train, 
                notes_test, 
                labels_test, 
                gold_labels_test,
                results_file,
                verbose=1):
    
    print('*'*80)
    print("Parameters (note: embed_size*10, latent_dim*64):\n", param)
    print('*'*80)
    
    start_time = time.time()
    
    # assign parameters
    up = int(param['up'])
    window_size = int(param['window_size'])
    embed_size = int(param['embed_size'] * 10)
    latent_dim = int(param['latent_dim'] * 64)
    dropout_rate = param['dropout_rate']
    epochs = param['epochs']
    category = param['category']
    max_features = 60000 #param['max_features']
    train_embed = True #param['train_embed']
    model_type = 'CuDNNLSTM' #param['model_type']
    
    # upsampling
    if up > 0:
        if verbose != 0: print('upsampling for %d times...' % (up))
        notes_train = [note + up * up_note for note, up_note in zip(notes_train, up_notes_train)]
        labels_train = [label + up * up_label for label, up_label in zip(labels_train, up_labels_train)]
    notes = notes_train + notes_test
    labels = labels_train + labels_test
    gold_labels = gold_labels_train + gold_labels_test
    
    # prepare features
    X_train_seq, X_test_seq, word_index = get_features(max_features, notes_train, notes_test, verbose=1)
    nb_words = min(max_features, len(word_index))

    # prepare embedding matrix
    if train_embed:
        if verbose != 0: print('preparing pretrained embedding matrix ...')
        w2v = Word2Vec(notes, size=embed_size, window=window_size, min_count=1, workers=4)
        embedding_index = dict(zip(w2v.wv.index2word, w2v.wv.vectors))
        embedding_matrix = get_embedding_matrix(embedding_index=embedding_index, 
                                                word_index=word_index, 
                                                max_features=max_features, 
                                                embed_size=embed_size)
        
    # prepare targets
    Y_train, Y_test, mlb, num_labels = get_targets(labels_train, labels_test, category, verbose=1)  

    # get rnn model
    model = get_rnn_model(nb_words=nb_words, 
                          num_labels=num_labels, 
                          embed_size=embed_size, 
                          latent_dim=latent_dim, 
                          model_type=model_type, 
                          embedding_matrix=embedding_matrix, 
                          dropout=dropout_rate, 
                          train_embed=train_embed)
    if verbose != 0: 
        print('model summary:')
        print(model.summary())
    
    # model compiling
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # model training
    if verbose != 0: print('\ntraining model ...')
    custevl = CustomEvaluation(validation_data=(X_test_seq, Y_test), interval=1)
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0005, patience=2, verbose=0, mode='auto')
    train_gen = data_generator(X_train_seq, Y_train)
    test_gen = data_generator(X_test_seq, Y_test)
    v = 1 if verbose != 0 else 0  
    hist = model.fit_generator(train_gen,
                                steps_per_epoch=len(Y_train),
                                epochs=epochs,
                                validation_data=test_gen,
                                validation_steps=len(Y_test),
                                callbacks=[custevl, earlystop],
                                verbose=v)

    # prediction of test data
    if verbose != 0: print('predicting test data ...')
    Y_pred = []
    for x in X_test_seq:
        x = np.array(x).reshape((1,-1))
        y_pred = np.squeeze(model.predict_on_batch(x))
        Y_pred.append(y_pred)
    Y_pred_concat = np.concatenate(Y_pred)
    Y_val = np.concatenate(Y_test)

    # confusion matrix 
    if verbose == 2: 
        cm = multilabel_confusion_matrix(Y_val, np.where(Y_pred_concat > 0.5, 1, 0))
        for i, j in zip(cm, mlb.classes_):
            print(j+':\n', i,'\n')

    # prepare gold label targets
    Y_gold_test, Y_gold_pred, gmlb = get_gold_label_targets(Y_pred, gold_labels, gold_labels_test, mlb, category=category, verbose=1) 

    # f1 scores for gold label
    f1 = f1_score(Y_gold_test, Y_gold_pred, average='micro')
    print('\nF1 Scores for global labels:\nALL (average="micro"):', f1)
    
    # confusion matrix for gold label
    if verbose == 2: 
        gcm = multilabel_confusion_matrix(np.concatenate(Y_gold_test), np.concatenate(Y_gold_pred))
        for i, j in zip(gcm, gmlb.classes_):
            print(j+':\n', i,'\n')
    
    # f1 score list
    if verbose == 2: 
        f1_all = f1_score(Y_gold_test, Y_gold_pred, average=None)
        for i, j in zip(f1_all, gmlb.classes_):
            print(j+': '+str(i))
    
    print('\n')
    
    elapsed_time = time.time() - start_time
    
    # save results
    with open(results_file,"a") as f:
        f.write("Parameters (note: embed_size*10, ltent_dim*64):\n" + str(param))
        f.write('\nF1 Scores for global labels(average="micro"): %.3f; Running time: %.1f\n' % (f1, elapsed_time))
          
    return f1

def bayes_opt(space):
    
    results_file = "cat_only_opt_results_" + time.strftime("%Y%m%d") + ".txt"
    
    param = {
            'up': space[0],               # Times of upsampling for training data
            'window_size': space[1],                # Window size for word2vec
            'embed_size': space[2],                # Length of the vector that we willl get from the embedding layer
            'latent_dim': space[3],               # Hidden layers dimension 
            'dropout_rate': space[4],             # Rate of the dropout layers
            'epochs': space[5],               # Max num of vocabulary
            'category': space[6]}                 # Number of epochs
            #'max_features': space[0],              # Is categoty labels
            #'train_embed': space[0],               # Using pre-made embedidng matrix as weight
            #'model_type': space[0]
            #}
                
    f1 = model_train(param, 
                    notes_train, 
                    labels_train, 
                    up_notes_train, 
                    up_labels_train, 
                    gold_labels_train, 
                    notes_test, 
                    labels_test, 
                    gold_labels_test,
                    results_file=results_file)
    
    return (-f1)    

if __name__ == "__main__":
    
    # loading data 
    if os.path.exists('loaded_data.dat'):
        
        with open('loaded_data.dat','rb') as f:
            notes_train = pickle.load(f)
            labels_train = pickle.load(f)
            up_notes_train = pickle.load(f)
            up_labels_train = pickle.load(f)
            gold_labels_train = pickle.load(f)
            notes_test = pickle.load(f)
            labels_test = pickle.load(f)
            gold_labels_test = pickle.load(f)
            
    else:
        
        notes_train_1, labels_train_1, up_notes_train_1, up_labels_train_1, gold_labels_train_1 = get_all_notes_labels('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set1') 
        notes_train_2, labels_train_2, up_notes_train_2, up_labels_train_2, gold_labels_train_2 = get_all_notes_labels('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set2') 

        notes_train = notes_train_1 + notes_train_2
        labels_train = labels_train_1 + labels_train_2
        up_notes_train = up_notes_train_1 + up_notes_train_2
        up_labels_train = up_labels_train_1 + up_labels_train_2
        gold_labels_train = gold_labels_train_1 + gold_labels_train_2

        notes_test, labels_test, _1, _2, gold_labels_test = get_all_notes_labels('/host_home/data/i2b2/2014/testing/testing-RiskFactors-Complete')

        with open('loaded_data.dat','wb') as f:
            pickle.dump(notes_train, f)
            pickle.dump(labels_train, f)
            pickle.dump(up_notes_train, f)
            pickle.dump(up_labels_train, f)
            pickle.dump(gold_labels_train, f)
            pickle.dump(notes_test, f)
            pickle.dump(labels_test, f)
            pickle.dump(gold_labels_test, f)

    # loading parameters space
    space = [Integer(5, 15, name='up'),
            Integer(3, 5, name='window_size'),
            Integer(3, 5, name='embed_size'),
            Integer(3, 5, name='latent_dim'),
            Real(0, 0.5, name='dropout_rate'),
            Integer(20, 21, name='epochs'),
            Categorical(['cat_only'], name='category')]
            #Integer(1, 60000, name='max_features'),
            #Categorical([True], name='train_embed'),
            #Categorical(['CuDNNLSTM'], name='model_type')]
    
    # initial parameters       
    x0 = [12, 3, 4, 5, 0.0, 21, None]
    
    # optimization
    #res_torch = gp_minimize(bayes_opt_torch, space, x0=x0, n_calls=50, verbose=True)
    res = gp_minimize(bayes_opt, space, x0=None, n_calls=50, verbose=True)
    
    # save opt results
    res_file = "cat_only_res_final_" + time.strftime("%Y%m%d") + ".txt"
    with open(res_file,"a") as f:
        f.write('opt by keras:\n%s\n' % str(res))
        
    # python command: python heart_no_pad_opt_comparison.py > heart_no_pad_opt_comparison.log

Using TensorFlow backend.


Iteration No: 1 started. Evaluating function at random point.
********************************************************************************
Parameters (note: embed_size*10, latent_dim*64):
 {'up': 9, 'window_size': 5, 'embed_size': 4, 'latent_dim': 3, 'dropout_rate': 0.34798445559982066, 'epochs': 21, 'category': 'cat_only'}
********************************************************************************
upsampling for 9 times...
preparing features ...
preparing pretrained embedding matrix ...
preparing targets ...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_

Epoch 1/20
790/790 [==============================] - 125s 159ms/step - loss: 0.0870 - acc: 0.9739 - val_loss: 0.0385 - val_acc: 0.9900
Adiitional val metrics: - ROC-AUC: 0.971663 - Log-Loss: 1.038037 - Hamming-Loss: 0.025190 - Subset-Accuracy: 0.851589 - F1-Score: 0.885898
Epoch 2/20
790/790 [==============================] - 125s 158ms/step - loss: 0.0567 - acc: 0.9823 - val_loss: 0.0282 - val_acc: 0.9919
Adiitional val metrics: - ROC-AUC: 0.985128 - Log-Loss: 0.675676 - Hamming-Loss: 0.014294 - Subset-Accuracy: 0.920808 - F1-Score: 0.935210
Epoch 3/20
790/790 [==============================] - 126s 159ms/step - loss: 0.0433 - acc: 0.9855 - val_loss: 0.0267 - val_acc: 0.9921
Adiitional val metrics: - ROC-AUC: 0.987360 - Log-Loss: 0.530800 - Hamming-Loss: 0.012669 - Subset-Accuracy: 0.930648 - F1-Score: 0.942430
Epoch 4/20
790/790 [==============================] - 125s 158ms/step - loss: 0.0343 - acc: 0.9883 - val_loss: 0.0263 - val_acc: 0.9927
Adiitional val metrics: - ROC-AUC: 0.98

preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, None, 40)          1799360   
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 640)         926720    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, None, 640)         0         
_________________________________________________________________
dense_11 (Dense)             (None, None, 300)         192300    
_________________________________________________________________
dense_12 (Dense)             (None, None, 9)           2709      
Total params: 2,921,089
Trainable params: 2,921,089
Non-trainab

790/790 [==============================] - 97s 123ms/step - loss: 0.0584 - acc: 0.9824 - val_loss: 0.0288 - val_acc: 0.9918
Adiitional val metrics: - ROC-AUC: 0.985689 - Log-Loss: 0.716008 - Hamming-Loss: 0.011881 - Subset-Accuracy: 0.935993 - F1-Score: 0.946060
Epoch 3/21
790/790 [==============================] - 96s 122ms/step - loss: 0.0442 - acc: 0.9857 - val_loss: 0.0265 - val_acc: 0.9925
Adiitional val metrics: - ROC-AUC: 0.989409 - Log-Loss: 0.517211 - Hamming-Loss: 0.010744 - Subset-Accuracy: 0.941294 - F1-Score: 0.951211
Epoch 4/21
790/790 [==============================] - 99s 125ms/step - loss: 0.0349 - acc: 0.9882 - val_loss: 0.0264 - val_acc: 0.9927
Adiitional val metrics: - ROC-AUC: 0.990135 - Log-Loss: 0.445243 - Hamming-Loss: 0.012276 - Subset-Accuracy: 0.933019 - F1-Score: 0.944182
Epoch 5/21
790/790 [==============================] - 98s 124ms/step - loss: 0.0293 - acc: 0.9899 - val_loss: 0.0271 - val_acc: 0.9929
Adiitional val metrics: - ROC-AUC: 0.990886 - Log-Loss

...

790/790 [==============================] - 147s 186ms/step - loss: 0.0529 - acc: 0.9830 - val_loss: 0.0285 - val_acc: 0.9918
Adiitional val metrics: - ROC-AUC: 0.984909 - Log-Loss: 0.624299 - Hamming-Loss: 0.014845 - Subset-Accuracy: 0.916664 - F1-Score: 0.932362
Epoch 4/21
790/790 [==============================] - 146s 185ms/step - loss: 0.0439 - acc: 0.9854 - val_loss: 0.0270 - val_acc: 0.9923
Adiitional val metrics: - ROC-AUC: 0.987363 - Log-Loss: 0.509639 - Hamming-Loss: 0.013762 - Subset-Accuracy: 0.924124 - F1-Score: 0.937370
Epoch 5/21
790/790 [==============================] - 147s 186ms/step - loss: 0.0364 - acc: 0.9875 - val_loss: 0.0275 - val_acc: 0.9926
Adiitional val metrics: - ROC-AUC: 0.988901 - Log-Loss: 0.435804 - Hamming-Loss: 0.012823 - Subset-Accuracy: 0.932249 - F1-Score: 0.941779
Epoch 6/21
790/790 [==============================] - 146s 185ms/step - loss: 0.0310 - acc: 0.9893 - val_loss: 0.0282 - val_acc: 0.9923
Adiitional val metrics: - ROC-AUC: 0.989356 - Log-

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_48 (Embedding)     (None, None, 50)          2249200   
_________________________________________________________________
bidirectional_48 (Bidirectio (None, None, 640)         952320    
_________________________________________________________________
spatial_dropout1d_48 (Spatia (None, None, 640)         0         
_________________________________________________________________
dense_95 (Dense)             (None, None, 300)         192300    
_________________________________________________________________
dense_96 (Dense)             (None, None, 9)           2709      
Total params: 3,396,529
Trainable params: 3,396,529
Non-trainab

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


preparing pretrained embedding matrix ...
preparing targets ...
model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_49 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_49 (Embedding)     (None, None, 50)          2249200   
_________________________________________________________________
bidirectional_49 (Bidirectio (None, None, 640)         952320    
_________________________________________________________________
spatial_dropout1d_49 (Spatia (None, None, 640)         0         
_________________________________________________________________
dense_97 (Dense)             (None, None, 300)         192300    
_________________________________________________________________
dense_98 (Dense)             (None, None, 9)           2709      
Total params: 3,396,529
Trainable params: 3,396,529
Non-trainab